# Project task 05:  Clustering users based on their preferences

In [48]:
import numpy as np
import scipy.sparse as sp

The goal of this task is to find groups of users with similar preferences using **Spectral clustering**. 
You are given a fragment of the Yelp social network, represented by an undirected weighted graph.
Nodes in the graph represent users.
If two users are connected by an edge of weight $w$, it means that they have both left positive reviews to the same $w$ restaurants.

Additionally, you are given a matrix `F` that encodes user preferences to different categories of restaurants. If `F[i, c] = 1`, then user `i` likes restaurants in category `c`.

You are allowed to use the imported functions (`eigsh`, `KMeans`, `normalize`).

## Load the data

* `N` = number of users (nodes in the graph)
* `C` = number of categories
* The graph is stored as a sparse adjacency matrix `A` (shape `[N, N]`).
* User preferences are stored in a matrix `F` (shape `[N, C]`). They will only be used for the final part of the assignment (Part 3)
* Name of each category is provided in the list `categories` (length `[C]`).

In [49]:
A = sp.load_npz('A.npz')
F = np.load('F.npy')
categories = np.load('categories.npy').tolist()

In [50]:
assert A.shape[0] == F.shape[0]
assert F.shape[1] == len(categories)

# 1. Implementing spectral clustering
## 1.1. Construct the graph Laplacian

First, we need to construct the Laplacian for the given graph. 

Given the **adjacency matrix** $A \in \mathbb{R}^{N \times N},$ we define the **degree matrix** $D \in \mathbb{R}^{N \times N}$ of an undirected graph as
$$D_{ij} =  \begin{cases}\sum_{k=1}^N A_{ik} & if \;\; i = j\\ 0 & if \;\; i \ne j\end{cases}$$

If our goal is to minimize the **ratio cut**, we will need to use the **unnormalized Laplacian**, defined as
$$L_{unnorm} = D - A.$$

If our goal is to minimze the **normalized cut**, we will need to use the **normalized Laplacian** (a.k.a. symmetrized Laplacian), defined as
$$L_{sym} = I - D^{-1/2}AD^{-1/2}$$

In [51]:
def construct_laplacian(A, norm_laplacian):
    """Construct Laplacian of a graph.
    
    Parameters
    ----------
    A : scipy.sparse.csr_matrix, shape [N, N]
        Adjacency matrix of the graph.
    norm_laplacian : bool
        Whether to construct the normalized graph Laplacian or not.
        If True, construct the normalized (symmetrized) Laplacian, L = I - D^{-1/2} A D^{-1/2}.
        If False, construct the unnormalized Laplacian, L = D - A.
        
    Returns
    -------
    L : scipy.sparse.csr_matrix, shape [N, N]
        Laplacian of the graph.
        
    """
    n = A.shape[0]
    diags = A.sum(axis=1)
    
    if norm_laplacian:
        diags =  1.0 / diags
        D_inv_sqrt = sp.diags(np.ravel(diags), 0, (n,n)).sqrt()
        L = sp.identity(n) - (D_inv_sqrt.dot(A)).dot(D_inv_sqrt)
    else:
        D = sp.diags(np.ravel(diags), 0, (n,n))
        L = D - A
    return L

## 1.2. Spectral embedding

Now, we have to compute the spectral embedding for the given graph.

In order to partition the graph into $k$ clusters, such that the desired cut (ratio or normalized) is minimized, we need to consider the $k$ eigenvectors corresponding to the $k$ smallest eigenvalues of the graph Laplacian.

Since the Laplacian matrix is sparse and symmetric, we can use the function `eigsh` from the `scipy.sparse.linalg` package in order to find eigendecomposition of $L$ (`eig` - eigendecomposition, `s` - sparse, `h`- Hermitian).
The function `eigsh` directly allows you to find the smallest / largest eigenvalues by specifying the `k` and `which` parameters. 

Keep in mind that the Laplacian matrix is always positive semi-definite when picking the appropriate value for the `which` parameter.

In [52]:
from scipy.sparse.linalg import eigsh

In [53]:
def spectral_embedding(A, num_clusters, norm_laplacian):
    """Compute spectral embedding of nodes in the given graph.
    
    Parameters
    ----------
    A : scipy.sparse.csr_matrix, shape [N, N]
        Adjacency matrix of the graph.
    num_clusters : int
        Number of clusters to detect in the data.
    norm_laplacian : bool, default False
        Whether to use the normalized graph Laplacian or not.
        
    Returns
    -------
    embedding : np.array, shape [N, num_clusters]
        Spectral embedding for the given graph.
        Each row represents the spectral embedding of a given node.
    
    """
    if (A != A.T).sum() != 0:
        raise ValueError("Spectral embedding doesn't work if the adjacency matrix is not symmetric.")
    laplacian = construct_laplacian(A, norm_laplacian)
    _, eigvecs = eigsh(laplacian, k=num_clusters, which="SM")
    return eigvecs

## 1.3. Determine the clusters based on the spectral embedding

You should use the K-means algorithm for assigning nodes to clusters, once the spectral embedding is computed.

One thing you should keep in mind, is that when using the **normalized Laplacian**, the rows of the embedding matrix **have to** be normalized to have unit $L_2$ norm.

In [54]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize

In [55]:
def spectral_clustering(A, num_clusters, norm_laplacian):
    """Perform spectral clustering on the given graph.
    
    Parameters
    ----------
    A : scipy.sparse.csr_matrix, shape [N, N]
        Adjacency matrix of the graph.
    num_clusters : int
        Number of clusters to detect in the data.
    norm_laplacian : bool, default False
        Whether to use the normalized graph Laplacian or not.
        
    Returns
    -------
    z_pred : np.array, shape [N]
        Predicted cluster indicators for each node.
        
    """
    embedding = spectral_embedding(A, num_clusters, norm_laplacian)
    if norm_laplacian:
        embedding = normalize(embedding)
    z_pred = KMeans(n_clusters=num_clusters).fit_predict(embedding)
    return z_pred

# 2. Quantitatively evaluate the results

In [56]:
def labels_to_list_of_clusters(z):
    """Convert predicted label vector to a list of clusters in the graph.
    This function is already implemented, nothing to do here.
    
    Parameters
    ----------
    z : np.array, shape [N]
        Predicted labels.
        
    Returns
    -------
    list_of_clusters : list of lists
        Each list contains ids of nodes that belong to the same cluster.
        Each node may appear in one and only one partition.
    
    Examples
    --------
    >>> z = np.array([0, 0, 1, 1, 0])
    >>> labels_to_list_of_clusters(z)
    [[0, 1, 4], [2, 3]]
    
    """
    return [np.where(z == c)[0] for c in np.unique(z)]

## 2.1. Compute ratio cut

Your task is to implement functions for computing the **ratio cut** and **normalized cut** for a given partition.

Ratio cut and normalized cut are defined on the slide 14 of the lecture slides.


The function `labels_to_list_of_clusters` can be helpful here.

In [57]:
def compute_ratio_cut(A, z):
    """Compute the ratio cut for the given partition of the graph.
    
    Parameters
    ----------
    A : scipy.sparse.csr_matrix, shape [N, N]
        Adjacency matrix of the graph.
    z : np.array, shape [N]
        Cluster indicators for each node.
    
    Returns
    -------
    ratio_cut : float
        Value of the cut for the given partition of the graph.
        
    """
    cluster_list = labels_to_list_of_clusters(z)
    ratio_cut = 0
    for cluster in cluster_list:
        normalizer = len(cluster)
        not_cluster = np.setdiff1d(range(len(z)), cluster)
        ratio_cut += (A[cluster, :][: ,not_cluster]).sum() / normalizer
    return ratio_cut

## 2.2. Compute normalized cut

**Important**: if a cluster only contains a single node, define its volume to be 1 to avoid division by zero errors.

In [58]:
def compute_normalized_cut(A, z):
    """Compute the normalized cut for the given partition of the graph.
    
    Parameters
    ----------
    A : scipy.sparse.csr_matrix, shape [N, N]
        Adjacency matrix of the graph.
    z : np.array, shape [N]
        Cluster indicators for each node.
    
    Returns
    -------
    norm_cut : float
        Value of the normalized cut for the given partition of the graph.
        
    """
    cluster_list = labels_to_list_of_clusters(z)
    norm_cut = 0
    for cluster in cluster_list:
        if len(cluster) == 1:
            vol = 1
        else:
            vol = np.sum(np.ravel(A.sum(axis=1))[cluster])
        not_cluster = np.setdiff1d(range(len(z)), cluster)
        norm_cut += (A[cluster, :][:, not_cluster]).sum() / vol
    return norm_cut

Notice, how using the unnormalized Laplacian leads to a much better ratio cut, while the normalized Laplacian leads to better normalized cut.

In [59]:
num_clusters = 6

In [60]:
norm_laplacian = False
z_unnorm = spectral_clustering(A, num_clusters, norm_laplacian)
print('When using L_unnorm:')
print(' ratio cut = {:.3f}'.format(compute_ratio_cut(A, z_unnorm)))
print(' normalized cut = {:.3f}'.format(compute_normalized_cut(A, z_unnorm)))
print(' sizes of partitions are: {}'.format([len(clust) for clust in labels_to_list_of_clusters(z_unnorm)]))

When using L_unnorm:
 ratio cut = 369.109
 normalized cut = 369.000
 sizes of partitions are: [3379, 1, 1, 1, 1, 1]


In [61]:
norm_laplacian = True
z_norm = spectral_clustering(A, num_clusters, norm_laplacian)
print('When using L_norm:')
print(' ratio cut = {:.3f}'.format(compute_ratio_cut(A, z_norm)))
print(' normalized cut = {:.3f}'.format(compute_normalized_cut(A, z_norm)))
print(' sizes of partitions are: {}'.format([len(clust) for clust in labels_to_list_of_clusters(z_norm)]))

When using L_norm:
 ratio cut = 5942.851
 normalized cut = 4.343
 sizes of partitions are: [572, 350, 742, 754, 577, 389]


# 3. Visualize the results

In the final part of the assignment, your task is to print out the 5 most popular types of restaurants visited by the users in each cluster.

In [62]:
def print_top_categories_for_each_cluster(top_k, z, F, categories):
    """Print the top-K categories among users in each cluster.
    For each cluster, the function prints names of the top-K categories,
    and number of users that like the respective category (separated by a comma).
    The function doesn't return anything, just prints the output.
    
    Parameters
    ----------
    top_k : int
        Number of most popular categories to print for each cluster.
    z : np.array, shape [N]
        Cluster labels.
    F : sp.csr_matrix, shape [N, C]
        Matrix that tells preferences of each user to each category.
        F[i, c] = 1 if user i gave at least one positive review to at least one restaurant in category c.
    categories : list, shape [C]
        Names of the categories.
        
    """
    clusters = labels_to_list_of_clusters(z)
    for cluster_num, cluster in enumerate(clusters):
        category_cluster_scores = F[cluster,:].sum(axis=0)
        most_popular_categories = np.argsort(category_cluster_scores)[-top_k:]
        print("Most popular categories in cluster {}".format(cluster_num))
        for i in range(top_k-1, -1, -1):
            print("\t {}, {}".format(categories[most_popular_categories[i]], category_cluster_scores[most_popular_categories[i]]))
        print("\n")

In [63]:
np.random.seed(2)
z_norm = spectral_clustering(A, num_clusters, True)
r = print_top_categories_for_each_cluster(5, z_norm, F, categories)

Most popular categories in cluster 0
	 Specialty Food, 356.0
	 Thai, 355.0
	 Breakfast & Brunch, 348.0
	 Japanese, 333.0
	 Ethnic Food, 330.0


Most popular categories in cluster 1
	 Seafood, 316.0
	 Mexican, 315.0
	 Sandwiches, 295.0
	 Japanese, 291.0
	 Breakfast & Brunch, 286.0


Most popular categories in cluster 2
	 Breakfast & Brunch, 665.0
	 Italian, 625.0
	 American (Traditional), 518.0
	 Sandwiches, 518.0
	 Pizza, 484.0


Most popular categories in cluster 3
	 Japanese, 535.0
	 Chinese, 447.0
	 Asian Fusion, 417.0
	 Sushi Bars, 414.0
	 Desserts, 411.0


Most popular categories in cluster 4
	 Japanese, 499.0
	 Breakfast & Brunch, 456.0
	 Sandwiches, 429.0
	 Italian, 414.0
	 Asian Fusion, 409.0


Most popular categories in cluster 5
	 Breakfast & Brunch, 636.0
	 Sandwiches, 528.0
	 Italian, 514.0
	 Pizza, 482.0
	 Coffee & Tea, 473.0


